In [1]:
from pypekit import Task
import pandas as pd


class IrisLoader(Task):
    input_types = ["source"]
    output_types = ["raw"]

    def run(self, _):
        from sklearn.datasets import load_iris
        iris = load_iris()
        iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
        iris_df['target'] = iris.target
        return iris_df


class TrainTestSplitter(Task):
    input_types = ["raw"]
    output_types = ["split"]

    def run(self, df):
        from sklearn.model_selection import train_test_split
        train_df, test_df = train_test_split(df, test_size=0.2)
        train_df['train'] = 1
        test_df['train'] = 0
        df = pd.concat([train_df, test_df], ignore_index=True)
        return df


class Scaler(Task):
    input_types = ["split"]
    output_types = ["processed"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        scaler = self.get_scaler()
        scaler.fit(X_train)

        X_scaled = scaler.transform(X)
        scaled_df = pd.DataFrame(data=X_scaled, columns=X.columns)
        scaled_df['target'] = df['target']
        scaled_df['train'] = df['train']

        return scaled_df

    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class MinMaxScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import MinMaxScaler
        return MinMaxScaler()


class StandardScaler(Scaler):
    def get_scaler(self):
        from sklearn.preprocessing import StandardScaler
        return StandardScaler()


class PCA(Task):
    input_types = ["split", "processed"]
    output_types = ["processed"]

    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        X_train = X[df['train'] == 1]

        from sklearn.decomposition import PCA
        pca = PCA(**self.kwargs)
        pca.fit(X_train)

        X_pca = pca.transform(X)
        pca_df = pd.DataFrame(data=X_pca, columns=[
                              f'PC[i+1]' for i in range(X_pca.shape[1])])
        pca_df['target'] = df['target']
        pca_df['train'] = df['train']

        return pca_df


class Classifier(Task):
    input_types = ["split", "processed"]
    output_types = ["processed", "predicted"]

    def run(self, df):
        X = df.drop(columns=['target', 'train'])
        y = df['target']
        X_train = X[df['train'] == 1]
        y_train = y[df['train'] == 1]

        classifier = self.get_classifier()
        classifier.fit(X_train, y_train)
        
        y_pred = classifier.predict(X)
        df['predicted'] = y_pred

        return df
    
    def get_scaler(self):
        raise NotImplementedError("Subclasses should implement this method.")


class LogisticRegression(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.linear_model import LogisticRegression
        return LogisticRegression(**self.kwargs)


class RandomForestClassifier(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.ensemble import RandomForestClassifier
        return RandomForestClassifier(**self.kwargs)


class SVC(Classifier):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def get_classifier(self):
        from sklearn.svm import SVC
        return SVC(**self.kwargs)


class Evaluator(Task):
    input_types = ["predicted"]
    output_types = ["sink"]

    def run(self, df):
        df_test = df[df['train'] == 0]
        return (df_test['target'] == df_test['predicted']).mean()

In [2]:
from pypekit import Repository, CachedExecutor

repository = Repository([
    IrisLoader,
    TrainTestSplitter,
    MinMaxScaler,
    StandardScaler,
    PCA,
    LogisticRegression,
    RandomForestClassifier,
    SVC,
    Evaluator
])

repository.build_tree()
print(repository.build_tree_string())

└── Root()
    └── IrisLoader()
        └── TrainTestSplitter()
            ├── MinMaxScaler()
            │   ├── PCA()
            │   │   ├── LogisticRegression()
            │   │   │   ├── RandomForestClassifier()
            │   │   │   │   ├── SVC()
            │   │   │   │   │   └── Evaluator()
            │   │   │   │   └── Evaluator()
            │   │   │   ├── SVC()
            │   │   │   │   ├── RandomForestClassifier()
            │   │   │   │   │   └── Evaluator()
            │   │   │   │   └── Evaluator()
            │   │   │   └── Evaluator()
            │   │   ├── RandomForestClassifier()
            │   │   │   ├── LogisticRegression()
            │   │   │   │   ├── SVC()
            │   │   │   │   │   └── Evaluator()
            │   │   │   │   └── Evaluator()
            │   │   │   ├── SVC()
            │   │   │   │   ├── LogisticRegression()
            │   │   │   │   │   └── Evaluator()
            │   │   │   │   └── Evaluator()
            │   │   │

In [3]:
pipelines = repository.build_pipelines()
pipelines[:10]

[Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), RandomForestClassifier(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), RandomForestClassifier(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), SVC(), RandomForestClassifier(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), LogisticRegression(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), RandomForestClassifier(), LogisticRegression(), SVC(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMaxScaler(), PCA(), RandomForestClassifier(), LogisticRegression(), Evaluator()]),
 Pipeline(tasks=[IrisLoader(), TrainTestSplitter(), MinMax

In [4]:
executor = CachedExecutor(pipelines, verbose=True)
results = executor.run()

Pipeline 1/144 completed. Runtime: 0.58s.
Pipeline 2/144 completed. Runtime: 0.58s.
Pipeline 3/144 completed. Runtime: 0.53s.
Pipeline 4/144 completed. Runtime: 0.47s.
Pipeline 5/144 completed. Runtime: 0.46s.
Pipeline 6/144 completed. Runtime: 0.53s.
Pipeline 7/144 completed. Runtime: 0.52s.
Pipeline 8/144 completed. Runtime: 0.52s.
Pipeline 9/144 completed. Runtime: 0.52s.
Pipeline 10/144 completed. Runtime: 0.51s.
Pipeline 11/144 completed. Runtime: 0.55s.
Pipeline 12/144 completed. Runtime: 0.45s.
Pipeline 13/144 completed. Runtime: 0.53s.
Pipeline 14/144 completed. Runtime: 0.52s.
Pipeline 15/144 completed. Runtime: 0.45s.
Pipeline 16/144 completed. Runtime: 0.52s.
Pipeline 17/144 completed. Runtime: 0.52s.
Pipeline 18/144 completed. Runtime: 0.50s.
Pipeline 19/144 completed. Runtime: 0.43s.
Pipeline 20/144 completed. Runtime: 0.50s.
Pipeline 21/144 completed. Runtime: 0.49s.
Pipeline 22/144 completed. Runtime: 0.49s.
Pipeline 23/144 completed. Runtime: 0.50s.
Pipeline 24/144 comp

In [5]:
for r in results.values():
    print(r)

{'output': np.float64(0.9666666666666667), 'runtime': 0.5790615909991175, 'tasks': ['IrisLoader()', 'TrainTestSplitter()', 'MinMaxScaler()', 'PCA()', 'LogisticRegression()', 'RandomForestClassifier()', 'SVC()', 'Evaluator()']}
{'output': np.float64(0.9666666666666667), 'runtime': 0.5767250099988814, 'tasks': ['IrisLoader()', 'TrainTestSplitter()', 'MinMaxScaler()', 'PCA()', 'LogisticRegression()', 'RandomForestClassifier()', 'Evaluator()']}
{'output': np.float64(0.9666666666666667), 'runtime': 0.5309084829996209, 'tasks': ['IrisLoader()', 'TrainTestSplitter()', 'MinMaxScaler()', 'PCA()', 'LogisticRegression()', 'SVC()', 'RandomForestClassifier()', 'Evaluator()']}
{'output': np.float64(0.9666666666666667), 'runtime': 0.4669725429994287, 'tasks': ['IrisLoader()', 'TrainTestSplitter()', 'MinMaxScaler()', 'PCA()', 'LogisticRegression()', 'SVC()', 'Evaluator()']}
{'output': np.float64(0.9666666666666667), 'runtime': 0.4643488539995815, 'tasks': ['IrisLoader()', 'TrainTestSplitter()', 'MinMa